In [30]:

from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

In [31]:
generator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
optimizer_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [32]:
# state
class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

In [33]:
from pydantic import BaseModel, Field
class TweetEvaluation(BaseModel):
    evaluation  : Literal["approved", "needs_improvement"] = Field(..., description="Evaluation of the tweet")
    feedback : str = Field(..., description="Feedback on the tweet")
    

In [34]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [35]:
def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]
    response = generator_llm.invoke(messages)
    return {'tweet' : response.content}

In [36]:
def evaluate_tweet(state: TweetState):
    # prompt
    messages = [
        SystemMessage(content="You are a social media expert evaluating tweets."),
        HumanMessage(content=f"""
Evaluate the following tweet for humor, originality, and engagement , dont approve halarious tweets, and provide feedback for improvement.:
Tweet: "{state['tweet']}"
                     
                     """)]
    response = structured_evaluator_llm.invoke(messages)    
    return {'evaluation': response.evaluation, 'feedback': response.feedback}

In [37]:
def optimize_tweet(state: TweetState):
    # prompt
    messages = [
        SystemMessage(content="You are a social media expert optimizing tweets."),
        HumanMessage(content=f"""Optimize the following tweet based on feedback:
original Tweet: "{state['tweet']}"  
Feedback: "{state['feedback']}"
Topic : "{state['topic']}"
 
 rewrite the tweet to be more engaging, funny, and original.

    """ ) ]
    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1
    if iteration >= state['max_iteration']:
        return {'tweet': response, 'iteration': iteration, 'max_iteration': state['max_iteration'], 'evaluation': 'approved', 'feedback': 'Final tweet after optimization.'}
    return {'tweet': response}




In [38]:
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return "approved"
    else:
        return "needs_improvement"

In [39]:
graph = StateGraph(TweetState)

graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate', route_evaluation, {"approved": END, "needs_improvement": 'optimize'})
graph.add_edge('optimize', 'evaluate')

In [40]:
workflow = graph.compile()


In [41]:
initial_state = TweetState({
    "topic": "AI and memes",
    "iteration": 1,
    "max_iteration": 4
})
workflow.invoke(initial_state)

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT